<a href="https://colab.research.google.com/github/cvs2010/rsp/blob/main/notebook_teste_machinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook criado com intuito de gerar um algoritmo de machine learning para predição do número de operadores a cada 15 minutos nas planilhas Horario RSP
Projeto Github RSP: https://github.com/cvs2010/rsp

# Observações:
- Tente o mínimo possível utilizar as bandas de GPU e TPU grátis do colab
- Esse arquivo é um arquivo de histórico, então tente trabalhar no seu GoogleColab em paralelo, somente suba as células para esse Notebook após finalizar todo o código ou gerar um checkpoint de análise do código (momento em que o código ainda não está pronto, mas necessita de análise do grupo)
- Muito cuidado com a edição desse arquivo e de outros arquivos da pasta do Drive: mais uma vez, tente trabalhar em paralelo, nunca utilize os arquivos originais, faça cópia de arquivos e trabalhe com seus arquivos localmente ou via cloud em seu login.
- O arquivo completo do DataSet dos horários de 2017 a abril/2023 está em https://github.com/cvs2010/rsp/releases/download/v0.0.1-alpha/horariosRSP_dataSet.zip

# Initial Configs
Configurações Iniciais

## Importação e Instalação de bibliotecas:
- bibliotecas disponíveis: import scipy, matplotlib, numpy, pandas, sklearn, seaborn, skimage, xgboost, lightgbm, catboost, tensorflow, keras, theano, pydot, graphviz, h5py, cv2, PIL, requests, bs4, lxml, html5lib, pyproj, shapely, geopandas, folium, nltk, spacy, textblob, gensim, glove_python, transformers, sentence_transformers, torchvision, torchtext, spacy, openai, tqdm, watermark, imageio, moviepy, simpleaudio, pygame, bokeh, plotly, dash, dash_core_components, dash_html_components, dash_bootstrap_components, ipywidgets, ipython, autoreload, pprint, inspect, warnings, logging, time, datetime, os, sys, math, random, json, re, hashlib, base64, uuid, collections, itertools, functools

In [1]:
#Importação de Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import numpy as np
import os
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from datetime import datetime
import urllib.request
import zipfile
import io

#Instalação de Bibliotecas
!pip install PyPDF2
!pip install spacy && python -m spacy download pt_core_news_sm
!pip install keras
!pip install tabula-py
!pip install pdfplumber
%pip install unidecode
from unidecode import unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-05 13:13:19.250746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 79.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.3 MB/s eta 0:00:00


Install 7zip reader libarchive

In [2]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.4.0-2ubuntu1.2_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Setting up libarchive-dev:amd64 (3.4.0-2ubuntu1.2) ...
Processing triggers for man-db (2.9.1-1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 6.3 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31644 sha256=7782ed62b4c65b126331c7cc7d9b2dcd549e9bff3a6ccfceed72b474f49ff49c
  Stored in directory: /root/.cache/pip/wheels/3a/94/d0/6cd83c8a80a4236fd4cb2a1fd846ecf72ab1e0ac238c5951c0
Successfully built libarchive


## Montando GoogleDrive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importação do DataFrame

In [4]:
df = pd.read_csv('https://github.com/cvs2010/rsp/raw/main/files/dataframe_backup.csv')

In [5]:
df

,horas,num_traf,dd_mm_aaaa,ds_fs_ff,00:00,00:15,00:30,00:45,01:00,01:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,D1Y1,1048,01/01/2017,DOM,4,2,3,2,2,2,...,6,2,9,4,9,8,4,4,4,6
1,D2Y1,962,02/01/2017,SEG,3,4,5,2,2,2,...,9,9,8,10,9,9,4,3,3,10
2,D3Y1,1150,03/01/2017,TER,2,4,3,2,2,2,...,10,8,4,7,7,5,3,3,2,7
3,D4Y1,685,04/01/2017,QUA,3,4,4,2,2,2,...,6,4,6,6,8,10,5,9,8,5
4,D5Y1,1017,05/01/2017,QUI,5,4,2,2,2,2,...,4,9,5,7,10,2,7,7,7,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820,D361Y5,913,27/12/2023,QUA,2,3,3,2,2,2,...,10,5,3,10,5,3,6,3,8,5
1821,D362Y5,1082,28/12/2023,QUI,3,4,2,2,2,2,...,5,2,9,3,4,4,9,4,2,6
1822,D363Y5,1154,29/12/2023,SEX,5,5,4,2,2,2,...,10,4,7,6,6,7,6,2,6,6
1823,D364Y5,963,30/12/2023,SÁB,4,5,3,2,2,2,...,10,5,2,8,6,6,2,6,7,3


## Importando DataSet
- Importando o DataSet HorarioTeste.zip do repositório do github
- Extraindo os arquivos na pasta /content/


In [6]:
# !wget https://github.com/cvs2010/rsp/releases/download/v0.0.1-alpha/horariosRSP_dataSet.zip
# !unzip horariosRSP_dataSet.zip -d /content/HorariosTeste

import urllib.request
import zipfile
import io

url = "https://github.com/cvs2010/rsp/releases/download/v0.0.1-alpha/horariosRSP_dataSet.zip"
with urllib.request.urlopen(url) as u:
    f = io.BytesIO(u.read())

with zipfile.ZipFile(f, 'r') as zip_ref:
    zip_ref.extractall('/content/HorariosTeste')

## Criando lista de arquivos da pasta do DataSet - HorariosTeste

In [7]:
import os

# Diretório onde os arquivos estão localizados
diretorio = '/content/HorariosTeste'

# Dicionário para armazenar os arquivos por extensão
arquivos_por_extensao = {}

# Percorre todos os arquivos em todas as subpastas do diretório
for pasta_atual, subpastas, arquivos in os.walk(diretorio):
    for nome_arquivo in arquivos:
        # Obtém a extensão do arquivo
        extensao = os.path.splitext(nome_arquivo)[1]
        # Adiciona o arquivo ao dicionário de acordo com a extensão
        if extensao in arquivos_por_extensao:
            arquivos_por_extensao[extensao].append(os.path.join(pasta_atual, nome_arquivo))
        else:
            arquivos_por_extensao[extensao] = [os.path.join(pasta_atual, nome_arquivo)]

# Imprime os arquivos por extensão
for extensao, arquivos in arquivos_por_extensao.items():
    print(extensao + ':')
    for arquivo in arquivos:
        print('\t' + arquivo)

# Dicionário para armazenar a contagem de cada extensão
contagem_por_extensao = {}

# Percorre todos os arquivos em todas as subpastas do diretório
for pasta_atual, subpastas, arquivos in os.walk(diretorio):
    for nome_arquivo in arquivos:
        # Obtém a extensão do arquivo
        extensao = os.path.splitext(nome_arquivo)[1]
        # Adiciona o arquivo ao dicionário de acordo com a extensão
        if extensao in arquivos_por_extensao:
            arquivos_por_extensao[extensao].append(os.path.join(pasta_atual, nome_arquivo))
        else:
            arquivos_por_extensao[extensao] = [os.path.join(pasta_atual, nome_arquivo)]
        # Incrementa a contagem da extensão
        if extensao in contagem_por_extensao:
            contagem_por_extensao[extensao] += 1
        else:
            contagem_por_extensao[extensao] = 1

# Imprime a contagem de cada extensão
for extensao, contagem in contagem_por_extensao.items():
    print(f"{extensao}: {contagem}")

A saída de streaming foi truncada nas últimas 5000 linhas.
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/25 JAN 2019 SEX TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/13 JAN 2019 DOM TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/16 JAN 2019 QUA TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/24 JAN 2019 QUI TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/08 JAN 2019 TER TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/09 JAN 2019 QUA TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/03 JAN QUI .ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/26 JAN 2019 SAB TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/23 JAN 2019 QUA TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/20 JAN 2019 DOM TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/14 JAN 2019 SEG TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/02 JAN 2019 QUA TARDE.ods
	/content/HorariosTeste/2019/01 JANEIRO/TARDE/12 JAN 2019 SAB TARDE.ods
	/content/Horar

## Resgatando a informação de data do nome do arquivo
Após a extração da data e transformação em uma string de data legível, iremos inserir essa informação na linha/coluna de data do dataframe
- O próximo passo é fazer com que ele leia todos os arquivos dentro da pasta HorariosTeste e processar um a um automaticamente de forma ordenada

In [8]:
# Para fazer esse processo são necessárias as bibliotecas os, re, unidecode e datetime

# Caminho do arquivo
filename = '/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx'

# Expressão regular para encontrar a data no nome do arquivo
date_regex = r'(\d{1,2})[-_./](\d{1,2})[-_./](\d{4})'

# Encontra a primeira data correspondente no nome do arquivo
match = re.search(date_regex, filename)

# Extrai a data no formato dd-mm-yyyy
def extract_date(filename):
    pattern = r'(\d{2})\s(\w{3})\s(\d{4})'
    match = re.search(pattern, filename)
    if match:
        day, month, year = match.groups()
        months = {
            'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
            'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
        }
        month = months[month.upper()]
        return f'{day}/{month}/{year}'
    else:
        return ""

# Rodando a função filename
formatted_date = filename
date = extract_date(formatted_date)
print(filename)
print(date)

# Extrai a data no formato dd-mm-yyyy
if match:
    day = match.group(1).zfill(2)
    month = match.group(2).zfill(2)
    year = match.group(3)
    formatted_date = f'{day}/{month}/{year}'
    print(formatted_date)
else:
    print('')

/content/HorariosTeste/2022_2023/Horário RSP - Backup - 12-04-2023 - WED.xlsx

12/04/2023


**# Protótipo do Machine Learning**

# **Descrição do Modelo de Machine Learning**
- Já com os dados tabulados em DataFrames, iremos dividir os dados em 70% para treinamento e 30% para teste (validação)

## **Primeiro Passo: Escolha do algoritmo de ML**
**- Qual algoritmo utilizaremos?**

  - Existem muitos tipos de algoritmos de machine learning, cada um com sua própria finalidade e propósito. Os principais tipos de algoritmos de machine learning são:

    - Aprendizado supervisionado: nesse tipo de algoritmo, o modelo é treinado com dados rotulados para fazer previsões precisas em novos dados rotulados. Isso pode incluir tarefas como classificação (prever a classe de um objeto) ou regressão (prever um valor numérico).

    - Aprendizado não supervisionado: esse tipo de algoritmo é usado quando não há rótulos nos dados. Em vez disso, o modelo tenta encontrar padrões ou agrupamentos nos dados. Isso pode incluir tarefas como clustering (agrupar objetos semelhantes) ou redução de dimensionalidade (reduzir o número de recursos).

    - Aprendizado por transferência: esse tipo de algoritmo usa conhecimento prévio aprendido em uma tarefa para ajudar em outra tarefa relacionada. Por exemplo, um modelo treinado para reconhecer rostos humanos pode ser transferido para reconhecer expressões faciais.

- Os principais algoritmos de aprendizado supervisionado são:
    - Regressão Linear: um algoritmo de regressão que tenta encontrar uma relação linear entre os recursos e o valor alvo, geralmente usado para tarefas de previsão numérica.

    - Árvores de Decisão: um algoritmo que divide o espaço de recurso em regiões menores e mais específicas, criando uma árvore de decisão que pode ser usada para prever a classe de um objeto ou valor numérico.

    - K-Nearest Neighbors (K-NN): um algoritmo que encontra os k objetos mais próximos de um novo objeto e usa a classe desses objetos para prever a classe do novo objeto.

    - Support Vector Machines (SVM): um algoritmo que tenta encontrar o hiperplano que separa as classes com a maior margem possível, usado para problemas de classificação binária e multiclasse.

    - Redes Neurais Artificiais: um algoritmo inspirado no funcionamento do cérebro humano, que usa várias camadas de neurônios para aprender uma representação hierárquica dos recursos e prever a classe ou valor alvo.


- Pelo tema do nosso projeto, acredito que o aprendizado supervisionado com algoritmo de regressão linear seria uma opção mais adequada para o problema.



# **Segundo Passo: Treinamento do modelo**
**- O objetivo aqui é ajustar os parâmetros do modelo de modo que ele possa fazer previsões precisas sobre novos dados.**

Na regressão linear simples, estamos tentando prever a variável de saída com base em uma única variável de entrada. No nosso caso, proponho que a variável de entrada (X) seja a quantidade total de horas trabalhadas no dia, e a variável de saída (y) é o número de operadores a cada 15 minutos do dia de trabalho.

Quando separamos nossos dados em conjuntos de treinamento e teste, precisamos garantir que o conjunto de treinamento contenha tanto as variáveis de entrada quanto as variáveis de saída. É por isso que criamos a matriz X_train com uma coluna e várias linhas, contendo os valores da variável de entrada para cada amostra do conjunto de treinamento. Já a matriz y_train contém os valores da variável de saída para cada amostra, e é unidimensional, com várias linhas.

Em resumo, X_train é uma matriz bidimensional com uma coluna, enquanto y_train é uma matriz unidimensional, pois estamos usando apenas uma variável de entrada na regressão linear simples.

## Separação dos dados de treinamento (X_train e y_train) e teste (X_test e y_test)



**MODELO DE REGRESSÃO LINEAR PARA ESTIMATIVA DO PARÂMETRO (Y) NÚMERO TOTAL DE HORAS TRABALHAS DO DIA DA SEMANA (SEG,TER,QUA,QUI,SEX,SAB,DOM,FERIADO)**

É possível utilizar redes neurais para estimar o número médio de horas trabalhadas em um determinado dia. Uma possível abordagem seria utilizar uma rede neural do tipo MLP (Multilayer Perceptron) para realizar a regressão.

A entrada da rede poderia ser um vetor numérico com as informações relevantes sobre o dia (por exemplo, a data, o dia da semana, o feriado ou não, etc.), e a saída seria um único número, representando a estimativa do número médio de horas trabalhadas nesse dia. Para treinar a rede, seria necessário um conjunto de dados de treinamento contendo pares (entrada, saída esperada) correspondentes a dias passados, em que o número médio de horas trabalhadas já é conhecido.

Uma vez treinada a rede, seria possível utilizá-la para realizar previsões para dias futuros, fornecendo como entrada as informações relevantes sobre o dia cuja previsão se deseja fazer. É importante notar que a qualidade das previsões dependerá da qualidade dos dados de treinamento e das características selecionadas como entrada da rede.

# **Terceiro Passo: Avaliar o desempenho do modelo**
**- Depois que o modelo é treinado, é importante avaliar o quão bem ele se comporta usando o conjunto de teste. Isso ajudará a determinar se o modelo está super ou subajustado e se há espaço para melhorias adicionais.**

# **Quarto Passo: Ajustar o modelo**
**- Se o desempenho do modelo não for satisfatório, é possível ajustá-lo modificando seus parâmetros ou experimentando um algoritmo de aprendizado de máquina diferente. Isso pode envolver a realização de mais experimentos e avaliações para encontrar a configuração ideal do modelo.**

# **Quinto Passo: Testar o modelo com previsões**
**- Uma vez que o modelo é treinado e ajustado, ele pode ser usado para fazer previsões em novos dados. Isso pode envolver a implementação do modelo em um aplicativo ou pipeline de produção para uso em um cenário do mundo real.**